In [1]:
import os
import numpy as np
import tensorflow as tf
#from data_generator_20240125 import process, tprocess
#from data_generator_20230522 import TrDataGenerator, ValDataGenerator
from tf_Dataset_20240219 import tf_TrDataset, tf_ValDataset

#os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '0'
#os.environ['TF_ENABLE_AUTO_MIXED_PRECISION_GRAPH_REWRITE'] = '0'
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

base_path = 'path that save the data'

train_image_path = 'path that save the training data'
val_image_path = 'path that save the validation data'
k = os.listdir(train_image_path)
path = []
for j in k:
    path.append(train_image_path + '/' + j)
vpath = []
k = os.listdir(val_image_path)
for j in k:
    vpath.append(val_image_path + '/' + j)

2024-02-19 08:40:35.266863: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-19 08:40:35.291072: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import keras.backend as K
from keras.losses import *
from keras.optimizers import Adam
from tensorflow.keras import mixed_precision
from keras.callbacks import ModelCheckpoint, EarlyStopping

from losses import dice_coefficient, generalized_dice_loss
from M4modelGnRes import build_model

mixed_precision.set_global_policy('mixed_float16')
tf.config.run_functions_eagerly(True)

d = []
l = []
vd = []
vl = []

batch_size = 1
vbatch_size = 1

steps_per_epoch = int(len(path)/batch_size)
steps_of_val = len(vpath)/vbatch_size

biparametric = True
if biparametric:
    input_channel_num = 2
else:
    input_channel_num = 1

model = build_model(input_shape=(None, None, None, input_channel_num), output_channels=1)

lr = 1e-4
#mpath = 'base model path'
model_base_path = 'directory to save model'
model_folder_name = 'model_folder_name'
spath = model_base_path + '/' + model_folder_name

os.makedirs(spath,exist_ok=True)

#model = load_model(mpath)
model.compile(optimizer = Adam(learning_rate=lr), loss = generalized_dice_loss, metrics = [dice_coefficient])
modelcheckpoint = ModelCheckpoint(spath + '/keras_M4_minTrLoss.h5',
                                  monitor='loss',save_best_only=True,mode='min')
modelcheckpoint1 = ModelCheckpoint(spath + '/keras_M4_minValLoss.h5',
                                  monitor='val_loss',save_best_only=True,mode='min')
modelcheckpoint2 = ModelCheckpoint(spath + '/keras_M4.h5')
modelcheckpoint3 = ModelCheckpoint(spath + '/keras_M4_maxTrDice.h5',
                                  monitor='dice_coefficient',save_best_only=True,mode='max')
modelcheckpoint4 = ModelCheckpoint(spath + '/keras_M4_maxValDice.h5',
                                  monitor='val_dice_coefficient',save_best_only=True,mode='max')
epoch_num = 2
history = model.fit(
    tf_TrDataset(path, batch_size, biparametric),
    #TrDataGenerator(path, batch_size, steps_per_epoch, biparametric),
    epochs=epoch_num,
    #steps_per_epoch=steps_per_epoch,
    validation_data=tf_ValDataset(vpath, vbatch_size, biparametric),
    #validation_data=ValDataGenerator(vpath, vbatch_size, steps_of_val, biparametric),
    #validation_steps = steps_of_val,
    validation_freq = 10,
    #max_queue_size = 20,
    #workers = 16,
    #use_multiprocessing = True,
    verbose = 2,
    callbacks=[modelcheckpoint, modelcheckpoint1, modelcheckpoint2, modelcheckpoint3, modelcheckpoint4])

d = history.history['dice_coefficient']
#d = history.history['tversky']
vd = history.history['val_dice_coefficient']
#vd = history.history['val_tversky']
l = history.history['loss']
vl = history.history['val_loss']

import pandas as pd    
df = pd.DataFrame({'trdice':d,
                   'trloss':l,
                  })
writer = pd.ExcelWriter(spath + '/TrDice&loss.xlsx')
df.to_excel(writer,'Sheet1',index=False)
writer._save()

df = pd.DataFrame({'vdice':vd,
                   'vloss':vl,
                  })
writer = pd.ExcelWriter(spath + '/ValDice&loss.xlsx')
df.to_excel(writer,'Sheet1',index=False)
writer._save()

print('Done')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4090, compute capability 8.9


2024-02-19 08:40:36.400930: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-19 08:40:36.404845: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-19 08:40:36.405120: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-19 08:40:36.405464: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-19 08:40:36.416234: I tensorflow/compile

Epoch 1/2


/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
2024-02-19 08:40:37.167111: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2486]
	 [[{{node Placeholder/_0}}]]
2024-02-19 08:40:37.167262: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and sh

KeyboardInterrupt: 